In [218]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import math
import torch.nn.functional as F
device = 'cpu'

In [172]:
a = (torch.arange(64)+1).view(8,8)
a

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29, 30, 31, 32],
        [33, 34, 35, 36, 37, 38, 39, 40],
        [41, 42, 43, 44, 45, 46, 47, 48],
        [49, 50, 51, 52, 53, 54, 55, 56],
        [57, 58, 59, 60, 61, 62, 63, 64]])

In [19]:
a[:-4,:4]

tensor([[ 1,  2,  3,  4],
        [ 9, 10, 11, 12],
        [17, 18, 19, 20],
        [25, 26, 27, 28]])

In [42]:
b = torch.zeros_like(a)
count = 1
for x in (slice(0,-4),slice(-4,None)):
    for y in (slice(0,-4),slice(-4,None)):
        b[x,y] = count 
        count += 1
b

tensor([[1, 1, 1, 1, 2, 2, 2, 2],
        [1, 1, 1, 1, 2, 2, 2, 2],
        [1, 1, 1, 1, 2, 2, 2, 2],
        [1, 1, 1, 1, 2, 2, 2, 2],
        [3, 3, 3, 3, 4, 4, 4, 4],
        [3, 3, 3, 3, 4, 4, 4, 4],
        [3, 3, 3, 3, 4, 4, 4, 4],
        [3, 3, 3, 3, 4, 4, 4, 4]])

In [128]:
# slice(2,-2), slice(2,-2) -> 1
# slice(2,-2),slice(-2,None) -> 2
# slice(2,-2),slice(None,2) -> 3
# slice(-2,None),slice(2,-2) -> 4
# slice(-2,None),slice(-2,None) -> 5
# slice(-2,None),slice(None,2) -> 6
# slice(None,2),slice(2,-2) -> 7
# slice(None,2),slice(-2,None) -> 8
# slice(None,2),slice(None,2) -> 9
count = 0
c = torch.zeros_like(a)
for i in (slice(None,4),slice(4,-2),slice(-2,None)):
    for j in (slice(None,4),slice(4,-2),slice(-2,None)):
        c[i,j] = count
        count+=1
c

tensor([[0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [3, 3, 3, 3, 4, 4, 5, 5],
        [3, 3, 3, 3, 4, 4, 5, 5],
        [6, 6, 6, 6, 7, 7, 8, 8],
        [6, 6, 6, 6, 7, 7, 8, 8]])

In [105]:
a[slice(None,4),slice(4,-2)]

tensor([[ 5,  6],
        [13, 14],
        [21, 22],
        [29, 30]])

In [123]:
shifted_a = torch.roll(a, shifts=(-2,-2), dims=(0, 1))

In [126]:
a

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29, 30, 31, 32],
        [33, 34, 35, 36, 37, 38, 39, 40],
        [41, 42, 43, 44, 45, 46, 47, 48],
        [49, 50, 51, 52, 53, 54, 55, 56],
        [57, 58, 59, 60, 61, 62, 63, 64]])

In [127]:
shifted_a

tensor([[19, 20, 21, 22, 23, 24, 17, 18],
        [27, 28, 29, 30, 31, 32, 25, 26],
        [35, 36, 37, 38, 39, 40, 33, 34],
        [43, 44, 45, 46, 47, 48, 41, 42],
        [51, 52, 53, 54, 55, 56, 49, 50],
        [59, 60, 61, 62, 63, 64, 57, 58],
        [ 3,  4,  5,  6,  7,  8,  1,  2],
        [11, 12, 13, 14, 15, 16,  9, 10]])

In [129]:
c

tensor([[0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [0, 0, 0, 0, 1, 1, 2, 2],
        [3, 3, 3, 3, 4, 4, 5, 5],
        [3, 3, 3, 3, 4, 4, 5, 5],
        [6, 6, 6, 6, 7, 7, 8, 8],
        [6, 6, 6, 6, 7, 7, 8, 8]])

In [134]:
H,W = 64,64
win = 4
img = torch.randn(1,3,H,W)
img = 

In [141]:
def partition(x:torch.tensor,window_size:int):
    B,C,H,W = x.shape
    x = x.view(B,H//window_size,window_size,W//window_size,window_size,3)
    return x.permute(0,1,3,2,4,5).contiguous().view(-1,win,win,C)

In [142]:
imgs = torch.randn(3,3,64,64)
out = partition(imgs,4)
out.shape

torch.Size([768, 4, 4, 3])

In [169]:
class Embedding(nn.Module):
    def __init__(self,patch_size=4,C=96):
        super().__init__()
        self.linear_embed = nn.Conv2d(3,C,kernel_size=patch_size,stride=patch_size)
        self.ln = nn.LayerNorm(C)
        self.relu = nn.ReLU()
    def forward(self,x):
        x = self.relu(self.ln(rearrange(self.linear_embed(x),'b c h w -> b (h w) c')))
        return x

Embedding()(torch.randn(1,3,224,224)).shape

torch.Size([1, 3136, 96])

In [156]:
class MergePatch(nn.Module):
    def __init__(self,C):
        super().__init__()
        self.linear = nn.Linear(4*C, 2*C)
        self.ln = nn.LayerNorm(2*C)
    def forward(self,x):
        h = w = int(math.sqrt(x.shape[1])/2)
        x = rearrange(x,'b (h s1 w s2) c -> b (h w) (s1 s2 c)',s1=2,s2=2,h=h,w=w)
        return self.ln(self.linear(x))

MergePatch(96)(torch.randn(1,3136,96)).shape

torch.Size([1, 784, 192])

In [219]:
class ShiftedWindowAttention(nn.Module):
    def __init__(self,embed_dim,num_heads,window_size=4,mask=False):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mask = mask
        self.window_size = window_size
        self.linear = nn.Linear(embed_dim,3 * embed_dim)
        self.ffd = nn.Linear(embed_dim,embed_dim)
        
    def forward(self,x):
        head_dim = self.embed_dim // self.num_heads
        h = w = int(math.sqrt(x.shape[1]))
        x = self.linear(x)
        x = rearrange(x,'b (h w) (c k) -> b h w c k',k=3,h=h,w=w)
        
        if self.mask:
            x = torch.roll(x,(-self.window_size//2,-self.window_size//2),dims=(1,2))
        
        x = rearrange(x,'b (h m1) (w m2) (nh he) k -> b nh h w (m1 m2) he k',nh=self.num_heads,he=head_dim,m1=self.window_size,m2=self.window_size)
        
        Q,K,V = x.chunk(3,dim=6)
        Q,K,V = map(lambda x : x.squeeze(-1),[Q,K,V])
        w = (Q @ K.transpose(4,5))/math.sqrt(head_dim)
        
        if self.mask:
            row_mask = torch.zeros((self.window_size**2,self.window_size**2)).to(device)
            row_mask[-self.window_size * (self.window_size//2):,:-self.window_size * (self.window_size//2)] = float('-inf')
            row_mask[:-self.window_size * (self.window_size//2),-self.window_size * (self.window_size//2):] = float('-inf')
            column_mask = rearrange(row_mask,'(r w1) (c w2) -> (w1 r) (w2 c)', w1=self.window_size, w2=self.window_size).to(device)
            w[:,:,-1,:] += row_mask
            w[:,:,:,-1] += column_mask
            
        attention = F.softmax(w,dim=-1) @ V
        x = rearrange(attention,'b nh h w (m1 m2) he -> b (h m1) (w m2) (nh he)',m1=self.window_size,m2=self.window_size)
        
        if self.mask:
            x = torch.roll(x,(self.window_size//2,self.window_size//2),dims=(1,2))
        
        x = rearrange(x,'b h w c -> b (h w) c')
        
        return self.ffd(x)
ShiftedWindowAttention(embed_dim=96,num_heads=4,mask=True)(torch.randn(1,56 * 56,96)).shape

torch.Size([1, 3136, 96])

In [220]:
a = torch.zeros((9,9))
a[:-3,-3:] = -math.inf
a[-3:,:-3] = -math.inf

In [208]:
column_mask = rearrange(a, '(r w1) (c w2) -> (w1 r) (w2 c)', w1=3, w2=3)
column_mask

tensor([[0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [-inf, -inf, 0., -inf, -inf, 0., -inf, -inf, 0.],
        [0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [-inf, -inf, 0., -inf, -inf, 0., -inf, -inf, 0.],
        [0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [0., 0., -inf, 0., 0., -inf, 0., 0., -inf],
        [-inf, -inf, 0., -inf, -inf, 0., -inf, -inf, 0.]])

In [209]:
a

tensor([[0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, 0., 0., 0.]])